In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import optimizers
import pandas as pd

import os

STEPS_PER_EPOCH = 500
BATCH_SIZE = 64
EPOCHS = 20
VAL_EPOCHS = 80

OPTIMIZER = optimizers.RMSprop(lr=1e-4)

LOGS_PATH = 'data/logs'
TRAIN_PATH = '../input/train'
TEST_PATH = '../input/test'
SAVE_PATH = 'data/predictions.csv'
WEIGHTS_PATH = 'data/weights/dogs_cats_net_{}_{}_{}.h5'.format(BATCH_SIZE, EPOCHS, OPTIMIZER)
CHECK_POINT_PREFIX = 'data/check_points/dogs_cats_net_{}_{}_{}'.format(BATCH_SIZE, STEPS_PER_EPOCH, OPTIMIZER)
CHECK_POINT_PATH = CHECK_POINT_PREFIX + '.{epoch:02d}-{val_loss:.4f}.pkl'
WEIGHTS_PATH

In [ ]:
model = Sequential()

model.add(Conv2D(32, 3, input_shape=(150, 150, 3), activation='relu', padding='same'))
model.add(Conv2D(32, 3, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(64, 3, activation='relu', padding='same'))
model.add(Conv2D(64, 3, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(128, 3, activation='relu', padding='same'))
model.add(Conv2D(128, 3, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(256, 3, activation='relu', padding='same'))
model.add(Conv2D(256, 3, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

for layer in model.layers:
    print('{:20s}'.format(layer.name), layer.input_shape, layer.output_shape)

model.compile(loss='binary_crossentropy',
              optimizer=OPTIMIZER,
              metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(150, 150),
    batch_size=BATCH_SIZE,
    class_mode='binary')

val_generator = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(150, 150),
    batch_size=BATCH_SIZE,
    class_mode='binary')

In [ ]:
if not os.path.exists(WEIGHTS_PATH):
    tb = TensorBoard(LOGS_PATH)
    checkpoint = ModelCheckpoint(CHECK_POINT_PATH, save_best_only=True)
    model.fit_generator(
        train_generator,
        steps_per_epoch=STEPS_PER_EPOCH,
        epochs=EPOCHS,
        verbose=1,
        callbacks=[tb, checkpoint],
        initial_epoch=0)
    model.save_weights(WEIGHTS_PATH)
else:
    model.load_weights(WEIGHTS_PATH)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(150, 150),
    batch_size=1,
    class_mode=None,
    shuffle=False)

raw_files = test_generator.filenames
raw_data = model.predict_generator(test_generator, 12500, verbose=1)